# **Redes Neurais e CNN: Identificação de Objetos**

Neste trabalho iremos fazer a identificação de um conjunto de dados de imagens de artigos de Zalando chamado Fashion-MNIST, que consiste em um conjunto de treinamento de 60.000 exemplos e um conjunto de teste de 10.000 exemplos. Cada exemplo é uma imagem em tons de cinza 28x28, associada a um rótulo de 10 classes.

Fonte: https://www.tensorflow.org/datasets/catalog/fashion_mnist?hl=pt-br

# **ATIVIDADE:**

**Descrição:**

Nesta atividade, você receberá um código de uma Rede Neural Artificial e uma Rede Neural Convolucional (CNN) implementada. Sua tarefa é analisar o código fornecido e fazer uma interpretação detalhada de cada linha, identificando e explicando as principais etapas, camadas e operações realizadas pela RNA e pela CNN.

**Instruções:**

1. Preencha o código com comentários em cada linha, explicando sua função e propósito.
2. Analise o código cuidadosamente para entender como a CNN é configurada.
3. Identifique e explique as camadas de convolução, camadas de pooling, camadas densas, funções de ativação, tamanhos dos filtros, número de neurônios, etc.
4. Identifique quais técnicas de regularização ou otimização formam utilizadas no código e explique como elas contribuem para o desempenho e a generalização do modelo.
5. Comente sobre a função de ativação utilizada na camada de saída.
6. Descreva os hiperparâmetros e ajustes do modelo, como a taxa de aprendizado, número de épocas de treinamento e o tratamento dos conjuntos de dados de treinamento e teste.
7. Faça uma avaliação descritiva dos resultados das Redes Neurais implementadas.



**Entregável:**

Prepare uma análise detalhada do código fornecido, destacando as principais características da CNN e como ela é configurada para a tarefa em questão. Se necessário, inclua observações sobre o potencial de melhoria ou otimização do modelo.

Esta atividade visa consolidar seu conhecimento sobre CNNs, ajudando a compreender como uma Rede Neural Convolucional é implementada e configurada para tarefas específicas.

In [ ]:
'''
importando o tensorflow e o keras
'''
import tensorflow as tf
from tensorflow import keras

In [ ]:
'''
importando mais algumas bibliotecas para auxiliar no desenvolvimento do trabalho
'''
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

## Importando base de dados

In [ ]:
'''
Aqui acessamos o dataset Fashion MNIST que já está incluso no Keras
o fashion_mnist é um conjunto de dados que vai ser usado para treinar e testar sistemas de aprendizado de máquina
'''
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
'''
Carregamento dos dados do fashion_mnist para a memória
e separação desses dados em dados de treino e dados de teste
'''
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
'''
Printando os formatos dos dados de treino e teste
'''
print(X_train_full.shape)
print(y_train_full.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
'''
Criação de um conjunto de validação a partir dos dados de treino
e normalização dos dados para que os valores fiquem entre 0 e 1
O código pega os primeiros 5000 exemplos do conjunto de treino para serem usados como validação
e o restante como conjunto de treino efetivo.
Por fim a separação dos rotulos é feita da mesma forma 
'''
# Transformar a escala para que os valores fiquem entre 0 e 1
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
'''
Definindo o nome das classes do dataset fashion_mnist
'''
nomes_classes = ["camisa/top", "calca", "casaco", "vestido", "jaqueta",
               "sandalia", "camiseta", "tenis", "bolsa", "bota"]

In [ ]:
'''
Acessa o sexto item do conjunto de treinamento e retorna o nome da classe correspondente
'''
nomes_classes[y_train[5]]

In [ ]:
'''
Acessa o sexto item do conjunto de teste
'''
X_train[5]

In [ ]:
'''
Definindo uma variável auxilicar para mostrar um exemplo do conjunto de treinamento
printa o nome da classe correspondente e mostra a imagem usando o matplotlib
é usado o nearest que significa que a imagem não será suavizada
'''
exemplo_n = 5
print(nomes_classes[y_train[exemplo_n]])
plt.imshow(X_train[exemplo_n], interpolation='nearest')
plt.show()

#  Rede Neural Artificial com Keras

In [ ]:
'''
Definindo uma RNA sequencial com camadas densas e dropout para evitar overfitting
Começa "achatando" a imagem 2D em um vetor único com a camada flatten
Depois ele empilha duas camadas ocultas densas com 300 e 100 neurônios que usam 
a função relu pra processar os dados, intercaladas com camadas de dropout 10%
que servem para desligar neurônios aleatórios e forçar a rede a aprender para que ela
não decore os exemplos
A rede termina com uma camada de saída de 10 neurônios usando softmax, responsável por
calcular a probabilidade final para cada uma das 10 categorias de roupas
então o summary() exibe um relatório com a estrutura criada
'''

nn = keras.models.Sequential()
nn.add(keras.layers.Flatten(input_shape=[28, 28]))
nn.add(keras.layers.Dense(300, activation="relu"))
nn.add(keras.layers.Dropout(rate=0.1))
nn.add(keras.layers.Dense(100, activation="relu"))
nn.add(keras.layers.Dropout(rate=0.1))
nn.add(keras.layers.Dense(10, activation="softmax"))
nn.summary()

### Compilando e treinando o modelo

In [ ]:
'''
configura e inicia o processo de aprendizado da rede
o comando %time serve como um cronômetro para medir a duração da execução
o método .compile() define a estratégia de estudo
ele escolhe a função de perda para calcular o erro da rede
define o otimizador sgd que vai ser usado para ajustar os pesos e corrigir esses erros
e determina monitorando a taxa de accuracy.
fit faz o treinamento usando os dados de validação (X_valid, y_valid) 
ao final de cada ciclo para testar se o aprendizado está realmente funcionando em dados novos
e é tudo salvo na variaável history_nn
'''

%time
nn.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history_nn = nn.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))

### Visualizar a performance

In [ ]:
'''
Análise visual dos dados de treino com o plt
'''
pd.DataFrame(history_nn.history).plot(figsize=(12, 8))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

### Avaliar o modelo

In [ ]:
'''
Executando a avaliação final da rede usando os dados de teste
exibe o erro final (loss) e a porcentagem de acertos
servindo como a a nota do desempenho do modelo
'''
# prints the loss and the accuracy
nn.evaluate(X_test, y_test,verbose=1)

###Realizar uma Previsão

In [ ]:
'''
realiza uma simulação de previsão com o modelo já treinado
primeiro ele seleciona as quatro primeiras imagens do conjunto de teste para servirem como dados novos
depois o método .predict() faz a rede analisar essas imagens e armazena o resultado em y_proba
'''
X_novo = X_test[:4]
y_proba = nn.predict(X_novo)
y_proba

In [ ]:
'''
analisa as porcentagens geradas anteriormente e seleciona o vencedor para cada imagem
ele identifica o índice da categoria que obteve a maior nota de confiança
depois o código imprime as classes e na última linha usa esses números como índices para buscar
e imprimir os nomes reais das roupas na lista de classes
'''
y_pred = np.argmax(nn.predict(X_novo), axis=-1)
print(y_pred)
print(np.array(nomes_classes)[y_pred])

### Verificar os resultados

In [ ]:
'''
Este trecho serve como o gabarito para verificar se a rede acertou
ele isola as respostas certas das quatro imagens analisadas e salva em y_novo
imprimindo esses códigos numéricos na tela
a última linha pega as previsões que o modelo fez (y_pred) e exibe os nomes correspondentes 
'''
y_novo = y_test[:4]
print(y_novo)
np.array(nomes_classes)[y_pred]

In [ ]:
'''
seleciona a terceira imagem do grupo de teste e imprime no console o nome da classe que
a rede acredita ser a correta, depois ele exibe a imagem real correspondente na tela
'''
# example_n = 201
print(nomes_classes[y_pred[2]])
plt.imshow(X_novo[2], interpolation='nearest')
plt.show()

# Treinando CNN com Keras

In [ ]:
'''
construção da CNN
começa definindo uma InputLayer explícita para imagens 28x28 em escala de cinza
depois inicia a extração de características por empilhamento
usando camadas de convolução para detectar padrões
seguidas de MaxPool2D para reduzir o tamanho da imagem e focar no que importa
aumentando progressivamente o valor dos filtros, que permite a rede aprender 
desde traços simples até formas complexas
o padding='same' faz com que a imagem não diminua de tamanho nas bordas durante as convoluções
'''
cnn = keras.models.Sequential([
keras.layers.InputLayer(input_shape = (28,28,1)),
keras.layers.Conv2D(64,5, activation='relu', padding='same', kernel_initializer='glorot_uniform'),
keras.layers.MaxPool2D(2),
keras.layers.Conv2D(128,3, activation='relu', padding='same'),
keras.layers.Conv2D(128,3, activation='relu', padding='same'),
keras.layers.MaxPool2D(2),
keras.layers.Conv2D(258,3, activation='relu', padding='same'),
keras.layers.Conv2D(258,3, activation='relu', padding='same'),
keras.layers.MaxPool2D(2),
keras.layers.Flatten(),
keras.layers.Dense(128, activation="relu"),
keras.layers.Dropout(rate=0.5),
keras.layers.Dense(64, activation="relu"),
keras.layers.Dropout(rate=0.5),
keras.layers.Dense(10, activation="softmax")])
    

In [ ]:
'''
Gera um relatório da estrutura da CNN criada
'''
cnn.summary()

In [ ]:
'''
responsável por adaptar o formato dos dados para que eles caibam na entrada da CNN
as camadas convolucionais precisam que os dados tenham 4 dimensões
(Quantidade de Imagens, Altura, Largura, Canais de Cor)
mas o dataset é carregado com 3 dimensões 
pra resolver isso é usado o .reshape() para adicionar o número 1 ao final das matrizes de treino
validação e teste
isso transforma o formato dos dados de (N, 28, 28) para (N, 28, 28, 1)
indicando que existe 1 canal de cor (preto e branco)
'''
print(X_train.shape)
X_train_new = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],1)
X_valid_new = X_valid.reshape(X_valid.shape[0], X_valid.shape[1], X_valid.shape[2],1)
X_test_new = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],1)
print(X_train_new.shape)

In [ ]:
'''
responsável por configurar as regras de aprendizado e dar a partida no treinamento da CNN
define a estratégia,  usando a função de perda sparse_categorical_crossentropy
escolhe o otimizador sgd para ajustar os pesos internos e foca na accuracy para medir o sucesso
.fit() treina o modelo usando os dados já redimensionados com o canal de cor
rodando por 20 epocas enquanto valida o desempenho passo a passo usando o conjunto de validação também ajustado (X_valid_new)
'''
cnn.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history_cnn = cnn.fit(X_train_new, y_train, epochs=20, validation_data=(X_valid_new, y_valid))

### Avaliar o modelo

In [ ]:
'''
faz a avaliação final de desempenho da CNN
'''
cnn.evaluate(X_test_new, y_test,verbose=1)

In [ ]:
'''Este trecho gera o gráfico de desempenho da CNN ao longo das 20 épocas de treinamento'''
pd.DataFrame(history_cnn.history).plot(figsize=(12, 8))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()